#Import Dependancies

In [1]:
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
import pandas as pd

import os

from PIL import Image
from tqdm import tqdm
import pdb

import torch
import torch.nn as nn
from torch.utils.data import Dataset, dataloader
from torchvision import datasets, models, transforms
from torchsummary import summary


In [2]:
torch.ops.load_library('/home/atharva/reduction/build/lib.linux-x86_64-3.10/custom_average.cpython-310-x86_64-linux-gnu.so')

In [3]:
#from google.colab import drive
#drive.mount("/content/gdrive")

In [4]:
root_dir = "/home/atharva/Adagrad/Challenge_images/"

image_folder = sorted(os.listdir(root_dir))
images = [i for i in image_folder]

df = pd.DataFrame()
df['path'] = [x for x in images]

df.to_csv('challenge_images.csv', header=None)

#Setting-up Dataloader

In [5]:
class dataset(Dataset):

  def __init__(self, root_dir, dataframe, transform = None):
    self.root_dir = root_dir
    self.dataframe = dataframe
    self.transform = transform

  def __len__(self):
    return(len(self.dataframe))

  def __getitem__(self, idx):

    image_path = os.path.join(self.root_dir, self.dataframe.iloc[idx, 0])
    image = Image.open(image_path)

    if self.transform:

      image = self.transform(image)
      return image

data_transforms = {
    'resize':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean = [0.485, 0.456, 0.406], 
                             std = [0.229, 0.224, 0.225])
    ])
}

image_datasets = {   
    'resize' : 
        dataset(
                dataframe = df,
                root_dir = root_dir,
                transform = data_transforms['resize']
            )
}

dataloaders = {
    'resize':
    torch.utils.data.DataLoader(image_datasets['resize'],
                                batch_size=64,
                                shuffle=False,
                                num_workers=8),  
}

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

#ReductionResnet Model

In [41]:
class ReductionResnet(nn.Module):
  def __init__(self):
    super(ReductionResnet, self).__init__()

    self.model = models.resnet18(pretrained=True)
    for param in self.model.parameters():
      param.require_grad = False

    self.primary_layers = nn.Sequential(*list(self.model.children())[:4])
    self.layer1 = nn.Sequential(*list(self.model.children())[4:5])
    self.avgpool1 = nn.AdaptiveAvgPool2d((1,1))
    self.layer2 = nn.Sequential(*list(self.model.children())[5:6])
    self.avgpool2 = nn.AdaptiveAvgPool2d((1,1))
    self.layer3 = nn.Sequential(*list(self.model.children())[6:7])
    self.avgpool3 = nn.AdaptiveAvgPool2d((1,1))
    self.layer4 = nn.Sequential(*list(self.model.children())[7:8])
    self.avgpool4 = nn.AdaptiveAvgPool2d((1,1))

  def forward(self, x):
    x = self.primary_layers(x)
    x = self.layer1(x)
    pool1 = self.avgpool1(x)
    x = self.layer2(x)
    pool2 = self.avgpool2(x)
    x = self.layer3(x)
    pool3 = self.avgpool3(x)
    x = self.layer4(x)
    pool4 = self.avgpool4(x)
    reduced = self.Reduction(pool1, pool2, pool3, pool4)
    return reduced

  def Reduction(self, output1, output2, output3, output4):

    self.pool = {}
    self.pool['avgpool1'] = output1
    self.pool['avgpool2'] = output2
    self.pool['avgpool3'] = output3
    self.pool['avgpool4'] = output4

    for layer in ['avgpool1', 'avgpool2', 'avgpool3','avgpool4']:
       b, c, h, w = self.pool[layer].shape
       self.pool[layer] = self.pool[layer].reshape(b, c*h*w)
        
    #embedding = torch.ops.custom_namespace.custom_average(self.pool['avgpool1'], self.pool['avgpool2'], self.pool['avgpool3'], self.pool['avgpool4'])

    return self.pool
      

In [42]:
model = ReductionResnet()
for image in dataloaders['resize']:
  output = model(image)

In [46]:
output['avgpool4'][0].shape

torch.Size([512])

In [38]:
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
value = cos(output[20], output[1])
value.item()

0.845493733882904

In [50]:
similairty = 0
similarity_idx = {}

for tensor1 in range(len(output['avgpool4'])):
    similarity_idx[tensor1] = 0
    
    for tensor2 in range(len(output['avgpool4'])):
        
        similarity = cos(output['avgpool4'][tensor1], output['avgpool4'][tensor2])
        if (similarity.item() > 0.75):
            similarity_idx[tensor1] += 1
            
            
    

image number:0
is similar to:0
is similar to:13
is similar to:45
image number:1
is similar to:1
is similar to:4
image number:2
is similar to:2
is similar to:7
is similar to:47
image number:3
is similar to:3
is similar to:39
image number:4
is similar to:1
is similar to:4
is similar to:8
is similar to:37
image number:5
is similar to:5
is similar to:28
image number:6
is similar to:6
is similar to:15
is similar to:25
is similar to:42
image number:7
is similar to:2
is similar to:7
is similar to:23
is similar to:37
image number:8
is similar to:4
is similar to:8
is similar to:37
image number:9
is similar to:9
image number:10
is similar to:10
image number:11
is similar to:11
image number:12
is similar to:12
image number:13
is similar to:0
is similar to:13
is similar to:21
is similar to:37
image number:14
is similar to:14
is similar to:18
is similar to:29
is similar to:37
is similar to:44
image number:15
is similar to:6
is similar to:15
image number:16
is similar to:16
is similar to:44
image nu

In [48]:
similarity_idx

{0: 3,
 1: 2,
 2: 3,
 3: 2,
 4: 4,
 5: 2,
 6: 4,
 7: 4,
 8: 3,
 9: 1,
 10: 1,
 11: 1,
 12: 1,
 13: 4,
 14: 5,
 15: 2,
 16: 2,
 17: 1,
 18: 2,
 19: 2,
 20: 1,
 21: 3,
 22: 2,
 23: 4,
 24: 2,
 25: 2,
 26: 1,
 27: 1,
 28: 2,
 29: 3,
 30: 1,
 31: 2,
 32: 2,
 33: 2,
 34: 2,
 35: 2,
 36: 2,
 37: 7,
 38: 2,
 39: 3,
 40: 1,
 41: 1,
 42: 2,
 43: 1,
 44: 3,
 45: 3,
 46: 5,
 47: 3,
 48: 2,
 49: 2}

#Exporting to Onnx

In [12]:
def reduction(g, input_64, input_128, input_256, input_512):
    return g.op("mydomain::custom_average", input_64, input_128, input_256, input_512)

from torch.onnx import register_custom_op_symbolic
register_custom_op_symbolic('custom_namespace::custom_average', reduction, 9)

dummy_input = torch.randn(50, 3, 224, 224)
model = ReductionResnet()

for image in dataloaders['resize']:
  output = model(image)

torch.onnx.export(model, 
                  dummy_input, 
                  'ReductionResnet.onnx',
                   opset_version=9,
                   verbose = True, 
                   input_names=['avgpool1', 'avgpool2', 'avgpool3', 'avgpool4'],
                   output_names=['embedding'],
                   custom_opsets={'custom_namespace': 2})

[W export.cpp:527] Warning: Custom opset domain: 'custom_namespace' provided is not used in the model. Please verify custom opset domain names. (function GraphEncoder)


graph(%avgpool1 : Float(50, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %219 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %220 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %222 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %223 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %225 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %226 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %228 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %229 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %231 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %232 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %234 : Float(128, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %235 : Float(128, strides=[1], requir